In [1]:
import quantecon
import numpy as np 
import matplotlib.pyplot as plt
import copy

In [50]:
class Person():
    def __init__(self,a,b):
        self.a = a
        self.b = b
    def get_utility(self,mu,var):
        return (self.a * mu - 0.5 * self.b * (var))

class Profession():
    def __init__(self,T,k,income_process):
        self.T = T
        self.k = k
        self.income_process = income_process

class Career():
    def __init__(self,A,C,G,mu0,sigma0,beta,T,k,a,b):
        if ((k<0) or (k>=T)):
            raise ValueError("k must be in the range [0, T-1]")
        
        if ((beta < 0) or (beta>1)):
            raise ValueError("beta must be in the range [0, 1]")
        
        self.A = A
        self.C = C
        self.G = G
        self.mu0 = mu0
        self.sigma0 = sigma0
        self.beta = beta 
        self.T = T
        self.k = k
        self.a = a
        self.b = b

        
    
    def construct_income_process(self):
        lss = quantecon.LinearStateSpace(A = self.A, C= self.C, G= self.G,
                                         mu_0=self.mu0,Sigma_0=(self.sigma0)**2)
        return lss
    
    def construct_profession(self,income_process):
        profession = Profession(self.T,self.k,income_process)
        return profession
    
    def get_unconditional_mean_PV0(self,profession):
        T = profession.T
        k = profession.k
        income_process = copy.deepcopy(profession.income_process)

        m = income_process.moment_sequence()
        discounted_mean = []

        for t in range(T):
            μ_x, μ_y, Σ_x, Σ_y = next(m)
            discounted_mean.append(float(μ_y) * (self.beta ** t))
        
        exp_PV0 = sum(discounted_mean[k:])
        return exp_PV0
    
    def get_unconditional_var_PV0(self,profession):
        T = profession.T
        k = profession.k
        income_process = copy.deepcopy(profession.income_process)

        m = income_process.moment_sequence()
        discounted_var = []
        for t in range(T):
            μ_x, μ_y, Σ_x, Σ_y= next(m)
            discounted_var.append(float(Σ_y) * (self.beta ** (2*t)))
        
        exp_var0 = sum(discounted_var[k:])
        return exp_var0
    
    def get_conditional_mean_PV0(self,profession):
        T = profession.T
        k = profession.k
        income_process = copy.deepcopy(profession.income_process)
        income_process.Sigma_0=np.zeros(income_process.Sigma_0.shape)  # set x0 to deterministic 

        m = income_process.moment_sequence()
        discounted_mean = []

        for t in range(T):
            μ_x, μ_y, Σ_x, Σ_y = next(m)
            discounted_mean.append(float(μ_y) * (self.beta ** (t)))
        
        exp_PV0 = sum(discounted_mean[k:])
        return exp_PV0
    
    def get_conditional_var_PV0(self,profession):
        T = profession.T
        k = profession.k
        income_process = copy.deepcopy(profession.income_process)
        income_process.Sigma_0=np.zeros(income_process.Sigma_0.shape)  # set x0 to deterministic 

        m = income_process.moment_sequence()
        discounted_var = []
        for t in range(T):
            μ_x, μ_y, Σ_x, Σ_y= next(m)
            discounted_var.append(float(Σ_y) * (self.beta ** (2*t)))
        
        exp_var0 = sum(discounted_var[k:])
        return exp_var0

    def construct_person(self):
    
        person = Person(self.a,self.b)
        return person
    
    def get_unconditional_EU(self,person,profession):
        mu = self.get_unconditional_mean_PV0(profession)
        var = self.get_unconditional_var_PV0(profession)

        EU = person.get_utility(mu,var)
        return EU
        

    def get_conditional_EU(self,person,profession):
        mu = self.get_conditional_mean_PV0(profession)
        var = self.get_conditional_var_PV0(profession)

        EU = person.get_utility(mu,var)
        return EU
        

In [51]:
A1 = np.array([[0.8, 0.2],
               [0.1, 0.9]])
C1 = np.eye(2)
G1 = np.array([[1.0, 0.0]])
mu_01 = np.array([1.0, 1.0])
Sigma_01 = np.eye(2)
beta = 0.95
T = 30
k1 = 20  # Years of schooling required for profession 1

# Profession 2
A2 = np.array([[0.9, 0.1],
               [0.2, 0.8]])
C2 = np.eye(2)
G2 = np.array([[1.2, -0.4]])
mu_02 = np.array([0.8, 1.2])
Sigma_02 = np.eye(2)
k2 = 5  # Years of schooling required for profession 2

# Profession 3
A3 = np.array([[0.85, 0.15],
               [0.05, 0.95]])
C3 = np.eye(2)
G3 = np.array([[1.5, 0.5]])
mu_03 = np.array([1.2, 0.8])
Sigma_03 = np.eye(2)
k3 = 3

In [53]:
test1 = Career(A=A1,C=C1,G=G1,mu0=mu_01,sigma0=Sigma_01,beta=beta,T=T,k=k1,a=1,b=0.2)
income1 = test1.construct_income_process()
prof1 = test1.construct_profession(income1)
person1 = test1.construct_person()

print("unconditional mean pv0: ",test1.get_unconditional_mean_PV0(prof1))
print("unconditional var pv0: ",test1.get_unconditional_var_PV0(prof1))

print("conditional mean pv0: ",test1.get_conditional_mean_PV0(prof1))
print("conditional var pv0: ",test1.get_conditional_var_PV0(prof1))

print("uncondtional EU: " ,test1.get_unconditional_EU(person1,prof1))
print("condtional EU: " ,test1.get_conditional_EU(person1,prof1))


unconditional mean pv0:  2.8769431693120904
unconditional var pv0:  11.809449296647085
conditional mean pv0:  2.8769431693120904
conditional var pv0:  11.339814649659559
uncondtional EU:  1.6959982396473818
condtional EU:  1.7429617043461345


The benefit of having separate classes for Profession and Person is that we can represent the Profession and Person individually for the discussion of their own characteristics

A profession can be 'uniquely' defined by its max career length `T`, education requirement `k`, and income stream (using `LinearStateSpace`). 

Similarly, a person can be described by his or her risk appetite `a` and `b`